In [2]:
# Import Library
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Create Spark Session
spark = SparkSession.builder.appName("Session").getOrCreate()

In [3]:
# Import Data
df = spark.read.option("mergeSchema",
                       "true").json("s3://emr-data-sync/segment-logs/*/*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------+-------+--------------------+-----+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+-------+
|           _metadata|         anonymousId|category|channel|             context|event|           messageId|           name|   originalTimestamp|           projectId|          properties|          receivedAt|              sentAt|           timestamp|              traits|    type|              userId|version|
+--------------------+--------------------+--------+-------+--------------------+-----+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+-------+
|{[Segment.io], []...|9d9a3a32-27f0-4af...|        | client|{{null, nu

In [5]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _metadata: struct (nullable = true)
 |    |-- bundled: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- bundledIds: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- unbundled: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- anonymousId: string (nullable = true)
 |-- category: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- context: struct (nullable = true)
 |    |-- campaign: struct (nullable = true)
 |    |    |-- content: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- medium: string (nullable = true)
 |    |    |-- mediumKMDForest: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- soruce: string (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- term: string (nullable = true)
 |    |-- ip: string (nullable = true)
 |    |-- library: struct (nul

In [6]:
new_df = df.withColumn("timestamp_cov", col("timestamp").cast("timestamp"))\
            .withColumn("utc_timestamp", to_utc_timestamp(col("timestamp_cov"), 'UTC'))\
            .withColumn("kst_timestamp", from_utc_timestamp(col("utc_timestamp"), 'Asia/Seoul'))\
            .withColumn("add_kst_timestamp", F.date_add(col("kst_timestamp"), 1))\
            .withColumn("ywd", F.date_sub(col("kst_timestamp"), 1))\
            .withColumn("yw", concat(year(col("add_kst_timestamp")), lit("-"), weekofyear(col("add_kst_timestamp")).cast("string")))\
            .withColumn("ym", date_format(col("timestamp"), "yyyy-MM").cast("string")).withColumn("ymd", date_format(col("timestamp"), "yyyy-MM-dd").cast("string"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
new_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------+-------+--------------------+-----+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+-------+--------------------+--------------------+--------------------+-----------------+----------+-------+-------+----------+
|           _metadata|         anonymousId|category|channel|             context|event|           messageId|           name|   originalTimestamp|           projectId|          properties|          receivedAt|              sentAt|           timestamp|              traits|    type|              userId|version|       timestamp_cov|       utc_timestamp|       kst_timestamp|add_kst_timestamp|       ywd|     yw|     ym|       ymd|
+--------------------+--------------------+--------+-------+--------------------+-----+--------------------+---------------+------------------

In [8]:
new_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _metadata: struct (nullable = true)
 |    |-- bundled: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- bundledIds: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- unbundled: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- anonymousId: string (nullable = true)
 |-- category: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- context: struct (nullable = true)
 |    |-- campaign: struct (nullable = true)
 |    |    |-- content: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- medium: string (nullable = true)
 |    |    |-- mediumKMDForest: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- soruce: string (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- term: string (nullable = true)
 |    |-- ip: string (nullable = true)
 |    |-- library: struct (nul

In [29]:
cov_new_df = new_df.selectExpr("anonymousid",
                               "context.userAgent AS context_useragent",
                               "context.ip AS context_ip",
                               "context.locale AS context_locale",
                               "context.page.url AS context_page_url",
                               "context",
                               "messageid",
                               "properties.path AS properties_path",
                               "properties.page AS properties_page",
                               "properties.button.name AS properties_button_name",
                               "properties.action.name AS properties_action_name",
                               "properties.action.hospital[0] AS properties_action_hospital_id",
                               "properties.action.hospital[1] AS properties_action_hospital_name",
                               "properties.button.hospital[0] AS properties_button_hospital_num",
                               "properties.button.hospital[1] AS properties_button_hospital_name",
                               "properties.properties.hospitalid AS properties_properties_hospitalid", 
                               "properties.properties.pagename AS properties_properties_pagename",
                               "properties.properties.routename AS properties_properties_routename",
                               "properties.properties.to AS properties_properties_to",
                               "properties",
                               "event",
                               "kst_timestamp AS timestamp",
                               "yw",
                               "ywd",
                               "ym",
                               "ymd",
                               "type",
                               "userId"
                              )

## 문제 -> 만약 컬럼이 조회되지 않는다면 작동이 안됨

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
cov_new_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- anonymousid: string (nullable = true)
 |-- context_useragent: string (nullable = true)
 |-- context_ip: string (nullable = true)
 |-- context_locale: string (nullable = true)
 |-- context_page_url: string (nullable = true)
 |-- context: struct (nullable = true)
 |    |-- campaign: struct (nullable = true)
 |    |    |-- content: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- medium: string (nullable = true)
 |    |    |-- mediumKMDForest: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- soruce: string (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- term: string (nullable = true)
 |    |-- ip: string (nullable = true)
 |    |-- library: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- version: string (nullable = true)
 |    |-- locale: string (nullable = true)
 |    |-- page: struct (nullable = true)
 |    |    |-- path: string (nullab

In [31]:
# Define the window
window = Window.partitionBy(cov_new_df.anonymousid).orderBy(cov_new_df.timestamp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# Add a column for inactivity time
cov_new_df = cov_new_df.withColumn("inactivity_time", 
                   (F.unix_timestamp(cov_new_df.timestamp) - F.lag(F.unix_timestamp(cov_new_df.timestamp)).over(window))/60)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# Filter for inactivity time greater than 30 minutes or null
cov_new_df = cov_new_df.filter((cov_new_df.inactivity_time > 30) | cov_new_df.inactivity_time.isNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# Add session_id column
cov_new_df = cov_new_df.withColumn("session_id", F.concat(cov_new_df.anonymousid, F.lit('-'), 
                                          F.row_number().over(window)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# Add next_session_start_at column
cov_new_df = cov_new_df.withColumn("next_session_start_at", 
                   F.lead(cov_new_df.timestamp).over(window))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Drop the inactivity_time column
cov_new_df = cov_new_df.drop("inactivity_time")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
cov_new_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- anonymousid: string (nullable = true)
 |-- context_useragent: string (nullable = true)
 |-- context_ip: string (nullable = true)
 |-- context_locale: string (nullable = true)
 |-- context_page_url: string (nullable = true)
 |-- context: struct (nullable = true)
 |    |-- campaign: struct (nullable = true)
 |    |    |-- content: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- medium: string (nullable = true)
 |    |    |-- mediumKMDForest: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- soruce: string (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- term: string (nullable = true)
 |    |-- ip: string (nullable = true)
 |    |-- library: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- version: string (nullable = true)
 |    |-- locale: string (nullable = true)
 |    |-- page: struct (nullable = true)
 |    |    |-- path: string (nullab

In [38]:
log_df = cov_new_df.selectExpr("session_id",
                               "anonymousid",
                              "timestamp AS session_start_at",
                              "next_session_start_at")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
log_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+---------------------+
|          session_id|         anonymousid|    session_start_at|next_session_start_at|
+--------------------+--------------------+--------------------+---------------------+
|0003566d-a0c0-477...|0003566d-a0c0-477...|2022-06-07 21:19:...|                 null|
|001dd17d-9481-406...|001dd17d-9481-406...|2022-06-29 21:28:...| 2022-06-29 22:45:...|
|001dd17d-9481-406...|001dd17d-9481-406...|2022-06-29 22:45:...| 2022-07-31 21:46:...|
|001dd17d-9481-406...|001dd17d-9481-406...|2022-07-31 21:46:...|                 null|
|002027ff-728d-496...|002027ff-728d-496...|2022-09-21 14:16:...| 2022-09-22 02:29:...|
+--------------------+--------------------+--------------------+---------------------+
only showing top 5 rows

In [40]:
log_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- session_id: string (nullable = true)
 |-- anonymousid: string (nullable = true)
 |-- session_start_at: timestamp (nullable = true)
 |-- next_session_start_at: timestamp (nullable = true)